In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from pprint import pprint
import gc
from collections import deque
from tqdm.notebook import tqdm
from time import sleep
import pickle
from datetime import datetime

In [2]:
pd.set_option("display.max_columns",None)

In [3]:
train = pd.read_csv('sales_train_cleaned.csv')
item_df = pd.read_csv('items.csv')
shops_df = pd.read_csv('shops.csv')

# Merging Mislabelled Shops

upon manually going through the data of the shops, discoveredthe following possible cases where the same shop have been given different shop_id, hence merging them together. The shops of interest are:
1. 0 => 57 (This is contentious)
2. 1 => 58 (This is contentious)
3. 10 => 11

In [4]:
train.loc[(train.shop_id == 0), 'shop_id'] = 57
train.loc[(train.shop_id == 1), 'shop_id'] = 58
train.loc[(train.shop_id == 10),'shop_id'] = 11

In [5]:
train['date'] = train['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))

train['day'] = train.date.apply(lambda x: int(x.strftime('%d')))
train['month'] = train.date.apply(lambda x: int(x.strftime('%m')))
train['year'] = train.date.apply(lambda x: int(x.strftime('%Y')))

# Monthly Sales

In [6]:
# Determine the monthly item count
monthly_sales = train.groupby(['shop_id','item_id','date_block_num'])['item_cnt_day'].sum().to_frame().reset_index()
monthly_sales.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)
monthly_sales

,shop_id,item_id,date_block_num,item_cnt_month
0,2,27,0,1.0
1,2,27,17,1.0
2,2,30,2,1.0
3,2,30,5,1.0
4,2,30,15,1.0
...,...,...,...,...
1609117,59,22164,27,2.0
1609118,59,22164,30,1.0
1609119,59,22167,9,1.0
1609120,59,22167,11,2.0


In [7]:
train_data = monthly_sales.drop(columns=['item_cnt_month'])
train_data

,shop_id,item_id,date_block_num
0,2,27,0
1,2,27,17
2,2,30,2
3,2,30,5
4,2,30,15
...,...,...,...
1609117,59,22164,27
1609118,59,22164,30
1609119,59,22167,9
1609120,59,22167,11


In [8]:
monthly_sales_seq = monthly_sales.pivot_table(index=['shop_id','item_id'], columns='date_block_num',
                                             values="item_cnt_month",fill_value=0)
monthly_sales_seq

date_block_num   0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  \
shop_id item_id                                                               
2       27        1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        30        0   0   1   0   0   1   0   0   0   0   0   0   0   0   0   
        31        0   4   1   1   0   0   0   0   0   0   0   0   0   0   0   
        32        0   0   0   0   0   0   0   0   0   0   0   0   1   0   1   
        33        1   0   0   0   0   0   0   0   0   0   2   1   1   0   0   
...              ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
59      22154     1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22155     0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   
        22162     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22164     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22167     0   0   0   0   0   0   0   0   0   1   0   2   0   0   0   

date_block_num   15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  \
shop_id item_id                                                               
2       27        0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   
        30        1   1   0   0   0   0   0   0   0   0   0   0   0   0   0   
        31        0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   
        32        0   0   0   1   0   2   2   0   2   0   0   1   0   0   0   
        33        0   0   0   0   0   1   0   0   0   0   1   0   1   1   0   
...              ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
59      22154     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22155     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22162     0   0   0   0   0   0   0   0   0   0   9   4   1   1   0   
        22164     0   0   0   0   0   0   0   0   0   0   2   1   2   0   0   
        22167     0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   

date_block_num   30  31  32  33  
shop_id item_id                  
2       27        0   0   0   0  
        30        0   0   0   0  
        31        0   0   0   1  
        32        0   1   0   0  
        33        1   0   1   0  
...              ..  ..  ..  ..  
59      22154     0   0   0   0  
        22155     0   0   0   0  
        22162     0   1   0   0  
        22164     1   0   0   0  
        22167     0   0   0   0  

[418908 rows x 34 columns]

# Features

## Average Monthly Shop Item Pair Price

In [9]:
avg_monthly_shop_item_price = train.groupby(['shop_id','item_id','date_block_num'])['item_price'].mean().reset_index()
avg_monthly_shop_item_price.rename(columns={'item_price':'avg_monthly_shop_item_price'}, inplace=True)
avg_monthly_shop_item_price

,shop_id,item_id,date_block_num,avg_monthly_shop_item_price
0,2,27,0,2499.0
1,2,27,17,498.0
2,2,30,2,359.0
3,2,30,5,399.0
4,2,30,15,169.0
...,...,...,...,...
1609117,59,22164,27,699.0
1609118,59,22164,30,699.0
1609119,59,22167,9,299.0
1609120,59,22167,11,299.0


In [10]:
price_data = avg_monthly_shop_item_price.pivot_table(index=['shop_id','item_id'], 
                                                     columns='date_block_num',
                                                     values='avg_monthly_shop_item_price',
                                                     fill_value=0)
price_data

date_block_num       0      1      2      3    4      5      6    7    8   \
shop_id item_id                                                             
2       27       2499.0    0.0    0.0    0.0  0.0    0.0    0.0  0.0  0.0   
        30          0.0    0.0  359.0    0.0  0.0  399.0    0.0  0.0  0.0   
        31          0.0  699.0  698.5  699.0  0.0    0.0    0.0  0.0  0.0   
        32          0.0    0.0    0.0    0.0  0.0    0.0    0.0  0.0  0.0   
        33        499.0    0.0    0.0    0.0  0.0    0.0    0.0  0.0  0.0   
...                 ...    ...    ...    ...  ...    ...    ...  ...  ...   
59      22154     999.0    0.0    0.0    0.0  0.0    0.0    0.0  0.0  0.0   
        22155       0.0    0.0    0.0    0.0  0.0    0.0  149.0  0.0  0.0   
        22162       0.0    0.0    0.0    0.0  0.0    0.0    0.0  0.0  0.0   
        22164       0.0    0.0    0.0    0.0  0.0    0.0    0.0  0.0  0.0   
        22167       0.0    0.0    0.0    0.0  0.0    0.0    0.0  0.0  0.0   

date_block_num      9      10     11     12   13     14     15      16     17  \
shop_id item_id                                                                 
2       27         0.0    0.0    0.0    0.0  0.0    0.0    0.0    0.00  498.0   
        30         0.0    0.0    0.0    0.0  0.0    0.0  169.0  169.00    0.0   
        31         0.0    0.0    0.0    0.0  0.0    0.0    0.0  415.92    0.0   
        32         0.0    0.0    0.0  119.0  0.0  149.0    0.0    0.00    0.0   
        33         0.0  199.0  199.0  199.0  0.0    0.0    0.0    0.00    0.0   
...                ...    ...    ...    ...  ...    ...    ...     ...    ...   
59      22154      0.0    0.0    0.0    0.0  0.0    0.0    0.0    0.00    0.0   
        22155      0.0    0.0    0.0    0.0  0.0    0.0    0.0    0.00    0.0   
        22162      0.0    0.0    0.0    0.0  0.0    0.0    0.0    0.00    0.0   
        22164      0.0    0.0    0.0    0.0  0.0    0.0    0.0    0.00    0.0   
        22167    299.0    0.0  299.0    0.0  0.0    0.0    0.0    0.00  299.0   

date_block_num      18   19     20     21   22     23   24     25     26  \
shop_id item_id                                                            
2       27         0.0  0.0    0.0    0.0  0.0    0.0  0.0    0.0    0.0   
        30         0.0  0.0    0.0    0.0  0.0    0.0  0.0    0.0    0.0   
        31         0.0  0.0    0.0    0.0  0.0    0.0  0.0    0.0    0.0   
        32       149.0  0.0  149.0  149.0  0.0  149.0  0.0    0.0  149.0   
        33         0.0  0.0  199.0    0.0  0.0    0.0  0.0  199.0    0.0   
...                ...  ...    ...    ...  ...    ...  ...    ...    ...   
59      22154      0.0  0.0    0.0    0.0  0.0    0.0  0.0    0.0    0.0   
        22155      0.0  0.0    0.0    0.0  0.0    0.0  0.0    0.0    0.0   
        22162      0.0  0.0    0.0    0.0  0.0    0.0  0.0  399.0  399.0   
        22164      0.0  0.0    0.0    0.0  0.0    0.0  0.0  749.0  749.0   
        22167      0.0  0.0    0.0    0.0  0.0    0.0  0.0    0.0    0.0   

date_block_num      27     28   29     30     31     32     33  
shop_id item_id                                                 
2       27         0.0    0.0  0.0    0.0    0.0    0.0    0.0  
        30         0.0    0.0  0.0    0.0    0.0    0.0    0.0  
        31         0.0    0.0  0.0    0.0    0.0    0.0  399.0  
        32         0.0    0.0  0.0    0.0  149.0    0.0    0.0  
        33       199.0  199.0  0.0  199.0    0.0  199.0    0.0  
...                ...    ...  ...    ...    ...    ...    ...  
59      22154      0.0    0.0  0.0    0.0    0.0    0.0    0.0  
        22155      0.0    0.0  0.0    0.0    0.0    0.0    0.0  
        22162    349.0  349.0  0.0    0.0  349.0    0.0    0.0  
        22164    699.0    0.0  0.0  699.0    0.0    0.0    0.0  
        22167      0.0    0.0  0.0    0.0    0.0    0.0    0.0  

[418908 rows x 34 columns]

In [11]:
# How to extract the row from the pivot table
price_data.loc[(2,27),:].values.reshape(-1,1)

array([[2499.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [ 498.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.]])

## Item Category ID

In [12]:
train_data = pd.merge(item_df,train_data, on=['item_id'])
train_data.drop(columns=['item_name'],inplace=True)
train_data = train_data.sort_values(by=['shop_id','date_block_num'])

In [13]:
item_cat_id = train_data.pivot_table(index=['shop_id','item_id'],
                                     columns='date_block_num',
                                     values='item_category_id',
                                     fill_value=0)
item_cat_id

date_block_num   0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  \
shop_id item_id                                                               
2       27       19   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        30        0   0  40   0   0  40   0   0   0   0   0   0   0   0   0   
        31        0  37  37  37   0   0   0   0   0   0   0   0   0   0   0   
        32        0   0   0   0   0   0   0   0   0   0   0   0  40   0  40   
        33       37   0   0   0   0   0   0   0   0   0  37  37  37   0   0   
...              ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
59      22154    37   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22155     0   0   0   0   0   0  37   0   0   0   0   0   0   0   0   
        22162     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22164     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22167     0   0   0   0   0   0   0   0   0  49   0  49   0   0   0   

date_block_num   15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  \
shop_id item_id                                                               
2       27        0   0  19   0   0   0   0   0   0   0   0   0   0   0   0   
        30       40  40   0   0   0   0   0   0   0   0   0   0   0   0   0   
        31        0  37   0   0   0   0   0   0   0   0   0   0   0   0   0   
        32        0   0   0  40   0  40  40   0  40   0   0  40   0   0   0   
        33        0   0   0   0   0  37   0   0   0   0  37   0  37  37   0   
...              ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   
59      22154     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22155     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
        22162     0   0   0   0   0   0   0   0   0   0  40  40  40  40   0   
        22164     0   0   0   0   0   0   0   0   0   0  37  37  37   0   0   
        22167     0   0  49   0   0   0   0   0   0   0   0   0   0   0   0   

date_block_num   30  31  32  33  
shop_id item_id                  
2       27        0   0   0   0  
        30        0   0   0   0  
        31        0   0   0  37  
        32        0  40   0   0  
        33       37   0  37   0  
...              ..  ..  ..  ..  
59      22154     0   0   0   0  
        22155     0   0   0   0  
        22162     0  40   0   0  
        22164    37   0   0   0  
        22167     0   0   0   0  

[418908 rows x 34 columns]

# Month Data

In [14]:
test = train.drop(columns=['day'],axis=1)

In [15]:
months = test.pivot_table(index=['shop_id','item_id'],
                 columns='date_block_num',
                 values='month',
                 fill_value=0)

## Year Data

In [16]:
years = test.pivot_table(index=['shop_id','item_id'],
                 columns='date_block_num',
                 values='year',
                 fill_value=0)

In [17]:
years

date_block_num     0     1     2     3   4     5     6   7   8     9     10  \
shop_id item_id                                                               
2       27       2013     0     0     0   0     0     0   0   0     0     0   
        30          0     0  2013     0   0  2013     0   0   0     0     0   
        31          0  2013  2013  2013   0     0     0   0   0     0     0   
        32          0     0     0     0   0     0     0   0   0     0     0   
        33       2013     0     0     0   0     0     0   0   0     0  2013   
...               ...   ...   ...   ...  ..   ...   ...  ..  ..   ...   ...   
59      22154    2013     0     0     0   0     0     0   0   0     0     0   
        22155       0     0     0     0   0     0  2013   0   0     0     0   
        22162       0     0     0     0   0     0     0   0   0     0     0   
        22164       0     0     0     0   0     0     0   0   0     0     0   
        22167       0     0     0     0   0     0     0   0   0  2013     0   

date_block_num     11    12  13    14    15    16    17    18  19    20    21  \
shop_id item_id                                                                 
2       27          0     0   0     0     0     0  2014     0   0     0     0   
        30          0     0   0     0  2014  2014     0     0   0     0     0   
        31          0     0   0     0     0  2014     0     0   0     0     0   
        32          0  2014   0  2014     0     0     0  2014   0  2014  2014   
        33       2013  2014   0     0     0     0     0     0   0  2014     0   
...               ...   ...  ..   ...   ...   ...   ...   ...  ..   ...   ...   
59      22154       0     0   0     0     0     0     0     0   0     0     0   
        22155       0     0   0     0     0     0     0     0   0     0     0   
        22162       0     0   0     0     0     0     0     0   0     0     0   
        22164       0     0   0     0     0     0     0     0   0     0     0   
        22167    2013     0   0     0     0     0  2014     0   0     0     0   

date_block_num   22    23  24    25    26    27    28  29    30    31    32  \
shop_id item_id                                                               
2       27        0     0   0     0     0     0     0   0     0     0     0   
        30        0     0   0     0     0     0     0   0     0     0     0   
        31        0     0   0     0     0     0     0   0     0     0     0   
        32        0  2014   0     0  2015     0     0   0     0  2015     0   
        33        0     0   0  2015     0  2015  2015   0  2015     0  2015   
...              ..   ...  ..   ...   ...   ...   ...  ..   ...   ...   ...   
59      22154     0     0   0     0     0     0     0   0     0     0     0   
        22155     0     0   0     0     0     0     0   0     0     0     0   
        22162     0     0   0  2015  2015  2015  2015   0     0  2015     0   
        22164     0     0   0  2015  2015  2015     0   0  2015     0     0   
        22167     0     0   0     0     0     0     0   0     0     0     0   

date_block_num     33  
shop_id item_id        
2       27          0  
        30          0  
        31       2015  
        32          0  
        33          0  
...               ...  
59      22154       0  
        22155       0  
        22162       0  
        22164       0  
        22167       0  

[418908 rows x 34 columns]